In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score

from datasetcovid import CovidDataset
from feedforwardnet import NeuralNetwork, NoRELU, TanhNet

d:\Programming Languages Package\Python\Python 3.10.1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Training Parameter

In [3]:
#device config
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#hyper params
input_size= 18
hidden_size= 16
num_epochs= 3
batch_size= 256
learning_rate= 0.001

# Dataset Loader

In [4]:
train_dataset=CovidDataset('lan1/train.csv')
test_dataset=CovidDataset('lan1/test.csv')

In [5]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=False)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [13]:
model=TanhNet(input_size,hidden_size)

#loss and optimizer
criterion=nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

# Training 

In [14]:
n_total_step=len(train_loader)
for epoch in range(num_epochs):
    for i, (feat,labels) in enumerate(train_loader):

        feat=feat.to(device)
        labels=labels.reshape(-1,1)
        labels=labels.to(device)
        #forward
        outputs=model(feat)
        loss=criterion(outputs,labels)
        #backward
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if i%100 == 0:
            print('Epoch ',epoch,'..Iter: ',i,'/',n_total_step,' LOSS: ',loss.item())

Epoch  0 ..Iter:  0 / 2795  LOSS:  2.0678021907806396
Epoch  0 ..Iter:  100 / 2795  LOSS:  0.32596608996391296
Epoch  0 ..Iter:  200 / 2795  LOSS:  0.31867533922195435
Epoch  0 ..Iter:  300 / 2795  LOSS:  0.25613945722579956
Epoch  0 ..Iter:  400 / 2795  LOSS:  0.21037307381629944
Epoch  0 ..Iter:  500 / 2795  LOSS:  0.23672939836978912
Epoch  0 ..Iter:  600 / 2795  LOSS:  0.2597908079624176
Epoch  0 ..Iter:  700 / 2795  LOSS:  0.26286768913269043
Epoch  0 ..Iter:  800 / 2795  LOSS:  0.20493176579475403
Epoch  0 ..Iter:  900 / 2795  LOSS:  0.3209874927997589
Epoch  0 ..Iter:  1000 / 2795  LOSS:  0.2820490002632141
Epoch  0 ..Iter:  1100 / 2795  LOSS:  0.20729400217533112
Epoch  0 ..Iter:  1200 / 2795  LOSS:  0.25741785764694214
Epoch  0 ..Iter:  1300 / 2795  LOSS:  0.16422303020954132
Epoch  0 ..Iter:  1400 / 2795  LOSS:  0.2390008419752121
Epoch  0 ..Iter:  1500 / 2795  LOSS:  0.1957041472196579
Epoch  0 ..Iter:  1600 / 2795  LOSS:  0.21854354441165924
Epoch  0 ..Iter:  1700 / 2795  L

Saving weights

In [7]:
torch.save(model.state_dict(), 'tmp.pth')

Loading weights

In [12]:
model = NoRELU(input_size,hidden_size)
model.load_state_dict(torch.load('tmp.pth'))
model.eval()

NoRELU(
  (l1): Linear(in_features=18, out_features=16, bias=True)
  (l2): Linear(in_features=16, out_features=1, bias=True)
)

# Testing

In [15]:
with torch.no_grad():
    n_correct=0
    n_samples=0
    vec_true=[]
    vec_pred=[]
    
    for i, (feat,labels) in enumerate(test_loader):
        
        #forward
        feat=feat.to(device)
        labels=labels.reshape(-1,1)
        vec_true+=labels
        labels=labels.to(device)
        outputs=model(feat)
        
        #result
        rst=[]
        for tsr in outputs:
            rst.append(tsr.round())
        vec_pred+=rst
        rst=torch.from_numpy(np.array(rst,dtype=np.float32))
        n_samples+=labels.shape[0]
        for i in range(labels.shape[0]):
            n_correct+=(labels[i]==rst[i])

    acc=100 * (n_correct)/n_samples
    print('-------ACC: ',float(acc))
    # Separate obj independently to avoid ERROR
    tmp=[]
    for tt in vec_true:
        tmp.append(float(tt))
    vec_true=np.array(tmp)
    tmp=[]
    for tt in vec_pred:
        tmp.append(float(tt))
    vec_pred=np.array(tmp)
    print('-------F1 score: ', f1_score(vec_true,vec_pred)*100)
    print('-------Pr score: ', precision_score(vec_true,vec_pred)*100)
    print('-------Re score: ', recall_score(vec_true,vec_pred)*100)

C:\Users\ACER\AppData\Local\Temp/ipykernel_21116/4184373153.py:21: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  rst=torch.from_numpy(np.array(rst,dtype=np.float32))
C:\Users\ACER\AppData\Local\Temp/ipykernel_21116/4184373153.py:21: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  rst=torch.from_numpy(np.array(rst,dtype=np.float32))


-------ACC:  90.7695541381836
-------F1 score:  42.28023658984295
-------Pr score:  62.166376776824805
-------Re score:  32.03325400995148
